In [ ]:
import numpy as np 
import pandas as pd 



### Naive SVM

In [ ]:
class SVM:
    def __init__(self,lr=1e-3,lambda_param=1e-2,n_iters=1000):
        self.lr = lr
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = 0
        
    def fit(self,X,y):
        
        samples,feat = X.shape
        y_ = np.where(y<=0,-1,1)
        
        self.w = np.zeros(feat)
        
        for _ in range(self.n_iters):
            for index,x in enumerate(X):
                
                cond = y_[index]* (np.dot(x,self.w)+self.b)>=1
                
                if cond:
                    dw = 2*self.lambda_param * self.w
                    db=0
                else:
                    dw = 2*self.lambda_param*self.w - np.dot(x,y_[index])
                    db = -y_[index]
                    
                    
            self.w -= self.lr * dw
            self.b -= self.lr * db
            
        
    def predict(self,X):
        op = np.dot(X,self.w)+self.b
        return np.sign(op)

In [ ]:
X = np.array([
    [2,2], [2,3], [3,2],    # class +1
    [-1,-1], [-2,-1], [-1,-2]  # class -1
])
y = np.array([1,1,1, -1,-1,-1])

# Train
clf = SVM(lr=0.001, lambda_param=0.01, n_iters=1000)
clf.fit(X, y)

# Predict
print("Predictions:", clf.predict(X))
print("True labels:", y)

### SVM with SMO

In [ ]:
class SVM_SMO:
    
    def __init__(self, C=1.0, tol=1e-3, max_passes=5):
        self.C = C          # regularization
        self.tol = tol      # tolerance for KKT violation
        self.max_passes = max_passes
        
    
    def fit(self,X,y):
        
        samples,features = X.shape
        y = np.where(y<=0,-1,1)
        self.alphas = np.zeros(samples)
        self.b = 0
        
        # Kernel Computation
        
        K = X @ X.T
        
        ##### SMO 
        
        passes = 0 
        while passes < self.max_passes:
            num_changed=0
            for i in range(samples):
                f_i = np.sum(self.alphas*y*K[:i])+self.b
                E_i  = f_i-y[i]
                
                # Misclassification and out of boundary
                if ((y[i]*E_i<-self.tol and self.alphas[i]<self.C) or (y[i]*E_i>self.tol and self.alphas[i]>0)):
                    
                    j = np.random.choice([ii for ii in range(samples) if ii!=i]) 
                    f_j = np.sum(self.alphas*y*K[:j])+self.b
                    E_j = f_j-y[j]
                    
                    
                    alpha_i_old,alpha_j_old = self.alphas[i],self.alphas[j]
                    
                    
                    # Sum should be zero
                    # Moving in opp direction
                    if y[i]!=y[j]:
                        L = max(0,self.alphas[j]-self.alphas[i])
                        H = max(self.C,self.C+self.alphas[j]-self.alphas[i])
                    else:
                        L = max(0,self.alphas[i]+self.alphas[j]-self.C)
                        H = min(self.C,self.alphas[i]+self.alphas[j])
                        
                    if L == H:
                        continue
                    
                    eta = 2*K[i,j]-K[i,i]-K[j,j]
                    
                    if eta>=0:
                        continue
                    
                    self.alphas[j] -=  (y[j]*(E_i-E_j))/eta
                    self.alphas[j] = np.clip(self.alphas[j],L,H)
                    
                    if abs(self.alphas[j]-alpha_j_old)<1e-5:
                        continue
                    
                    self.alphas[i] += y[i]*y[j]*(alpha_j_old-self.alphas[j]) 
                    
                    b1 = self.b - E_i \
                         - y[i] * (self.alphas[i] - alpha_i_old) * K[i, i] \
                         - y[j] * (self.alphas[j] - alpha_j_old) * K[i, j]
                    b2 = self.b - E_j \
                         - y[i] * (self.alphas[i] - alpha_i_old) * K[i, j] \
                         - y[j] * (self.alphas[j] - alpha_j_old) * K[j, j]
                      
                    if 0 < self.alphas[i] < self.C:
                        self.b = b1
                    elif 0 < self.alphas[j] < self.C:
                        self.b = b2
                    else:
                        self.b = 0.5 * (b1 + b2)
                        
                    num_changed+=1
                    
            passes = passes + 1 if num_changed == 0 else 0
            
        self.w = np.sum((self.alphas * y)[:, None] * X, axis=0)
        
    def project(self,X):
        return X @ self.w + self.b
    
    def predict(self,X):
        return np.sign(self.project(X))

                    

In [ ]:
# Tiny dataset
X = np.array([[2,2],[2,3],[3,2],[-1,-1],[-2,-1],[-1,-2]])
y = np.array([1,1,1,-1,-1,-1])

clf = SVM_SMO(C=1.0, max_passes=10)
clf.fit(X, y)

print("Weights:", clf.w)
print("Bias:", clf.b)
print("Predictions:", clf.predict(X))
